<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/PI1_MLOps_videojuegos/blob/main/JupyterNotebooks/01d_Feature_eng.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Preparación de conjuntos de datos para consultas

En esta jupyter notebook se preparan los conjuntos de datos necesarios para hacer las consultas en la API.

## Importaciones

In [75]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Extracción de los conjuntos de datos

Se extrean los datos de `df_reviews` que contiene la información relacionada a los usuarios que realizaron review de los juegos que consumen.

In [47]:
df_reviews = pd.read_csv('data/user_review_limpio.csv', encoding='utf-8')

Se extrean los datos de `df_games` que contiene la información relacionada a los juegos que tiene la plataforma.


In [48]:
df_games = pd.read_csv('data/steam_games_limpio.csv', encoding='utf-8')

Se extrean los datos de `df_items` que contiene la información relacionada al consumo de los juegos por parte de los usuarios.

In [49]:
df_items = pd.read_csv('data/user_items_limpio.csv', encoding='utf-8')

## Análisis de sentimientos

Se pide crear una nueva columna llamada 'sentiment_analysis' que reemplace a 'reviews_review' donde se realice un análisis de sentimiento de los comentarios con la siguiente escala:

* 0 si es malo,
* 1 si es neutral o esta sin review
* 2 si es positivo.

Dado que el objetivo de este proyecto es realizar una prueba de concepto, consiguiendo un producto mínimo viable, se realiza un análisis de sentimiento básico utilizando TextBlob que es una biblioteca de procesamiento de lenguaje natural (NLP) en Python. El objetivo de esta metodología es asignar un valor numérico a un texto, en este caso a los comentarios que los usuarios dejaron para un juego determinado, para representar si el sentimiento expresado en el texto es negativo, neutral o positivo. 

Esta metodología toma una revisión de texto como entrada, utiliza TextBlob para calcular la polaridad de sentimiento y luego clasifica la revisión como negativa, neutral o positiva en función de la polaridad calculada. En este caso, se consideraron las polaridades por defecto del modelo, el cuál utiliza umbrales -0.2 y 0.2, siendo polaridades negativas por debajo de -0.2, positivas por encima de 0.2 y neutrales entre medio de ambos.

In [50]:
df_reviews['sentiment_analysis'] = df_reviews['reviews_review'].apply(utils.analisis_sentimiento)
df_reviews.head()

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,1
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,1
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,Formato inválido,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013-10-14,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014-04-15,1


Se revisan algunos ejemplos para cada una de las clases de sentimiento.

In [51]:
utils.ejemplos_review_por_sentimiento(df_reviews['reviews_review'], df_reviews['sentiment_analysis'])

Para la categoría de análisis de sentimiento 0 se tienen estos ejemplos de reviews:
Review 1: This game is Marvellous.
Review 2: Killed the Emperor, nobody cared and got away with it. Accidentally killed a chicken and everybody decided to gang up on me. 10/10
Review 3: This Game Doesn't Work


Para la categoría de análisis de sentimiento 1 se tienen estos ejemplos de reviews:
Review 1: Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.
Review 2: I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it's title, this is easily one of my GOTYs. You don't get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can't 360 noscope your friends, but what you can do is show them up with your bad 

Finalmente, se elimina la columna de 'reviews_review'.

In [52]:
df_reviews = df_reviews.drop(columns=['reviews_review'])
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'reviews_date', 'sentiment_analysis'],
      dtype='object')

In [53]:
utils.verificar_tipo_datos(df_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.0,0.0,0
1,user_url,[<class 'str'>],100.0,0.0,0
2,reviews_item_id,[<class 'int'>],100.0,0.0,0
3,reviews_helpful,[<class 'str'>],100.0,0.0,0
4,reviews_recommend,[<class 'bool'>],100.0,0.0,0
5,reviews_date,[<class 'str'>],100.0,0.0,0
6,sentiment_analysis,[<class 'int'>],100.0,0.0,0


## Cantidad de dinero gastado por usuario y cantidad de items consumidos

Se crea el dataframe 'df_gastos_items' que contiene por cada usuario el gasto en videosjuegos y la cantidad de items consumidos. Para ello se realian una serie de pasos creando algunos dataframe auxiliares para luego unir los resultados.

In [54]:
# Se extraen las columnas 'items_count', 'user_id' e 'item_id'
df_gastos_items = df_items[['items_count', 'user_id', 'item_id']]
df_gastos_items

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20
2,277,76561197970982479,30
3,277,76561197970982479,40
4,277,76561197970982479,50
...,...,...,...
5094100,7,76561198329548331,346330
5094101,7,76561198329548331,373330
5094102,7,76561198329548331,388490
5094103,7,76561198329548331,521570


Se reserva el dataframe anterior y se realiza un nuevo dataframe auxiliar, pero con el precio de cada juego.

In [55]:
price_juegos = df_games[['price', 'id']]
# Elimina los duplicados
price_juegos = price_juegos.drop_duplicates(subset='id', keep='first')
# Se renombra 'id' por 'item_id' para unir mas adelante
price_juegos = price_juegos.rename(columns={'id':'item_id'})
price_juegos

,price,item_id
0,4.99,761140
5,0.00,643980
9,0.00,670290
14,0.99,767400
17,3.99,772540
...,...,...
71535,1.99,745400
71539,1.99,773640
71543,4.99,733530
71546,1.99,610660


Finalmente, se unen los dos dataframe auxiliares para conformar un dataframe final llamado `df_gastos_items`.

In [56]:
df_gastos_items = df_gastos_items.merge(price_juegos, on='item_id', how='left')
df_gastos_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094100,7,76561198329548331,346330,0.00
5094101,7,76561198329548331,373330,NaN
5094102,7,76561198329548331,388490,0.00
5094103,7,76561198329548331,521570,0.00


Se verifican los tipos de datos y si hay nulos.

In [57]:
utils.verificar_tipo_datos(df_gastos_items)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,items_count,[<class 'int'>],100.00,0.00,0
1,user_id,[<class 'str'>],100.00,0.00,0
2,item_id,[<class 'int'>],100.00,0.00,0
3,price,[<class 'float'>],81.59,18.41,937657


Se observan valores nulos de 'price' por lo que se explora la columna.

In [58]:
df_gastos_items[df_gastos_items['price'].isnull()]

,items_count,user_id,item_id,price
13,277,76561197970982479,34440,NaN
26,277,76561197970982479,9340,NaN
27,277,76561197970982479,228200,NaN
37,277,76561197970982479,17340,NaN
51,277,76561197970982479,23120,NaN
...,...,...,...,...
5094089,177,76561198326700687,508380,NaN
5094091,177,76561198326700687,510660,NaN
5094095,177,76561198326700687,519170,NaN
5094101,7,76561198329548331,373330,NaN


Hay cerca de un 19% de juegos que no tienen precio. Como no se cuenta con información acerca de los mismos, se asume que los que no tienen precio es porque son gratuitos y se los rellena con 0.0.

In [59]:
df_relleno = df_gastos_items['price'].fillna(0.0)
# Se borra la columna original y se concatena la columna rellena con todo el dataframe
df_gastos_items = pd.concat([df_gastos_items.drop('price', axis=1), df_relleno], axis=1)
df_gastos_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094100,7,76561198329548331,346330,0.00
5094101,7,76561198329548331,373330,0.00
5094102,7,76561198329548331,388490,0.00
5094103,7,76561198329548331,521570,0.00


Finalmente, se elimina la columna 'item_id' y se agrupa por usuario para sumar la cantidad gastada por usuario.

In [60]:
# Se elimina 'item_id'
df_gastos_items = df_gastos_items.drop('item_id', axis=1)
df_gastos_items.columns

Index(['items_count', 'user_id', 'price'], dtype='object')

In [61]:
# Se agrupa por usuario sumando los precios de los juegos consumidos
df_gastos_items_group = df_gastos_items.groupby('user_id')['price'].sum().reset_index()
df_gastos_items_group

,user_id,price
0,--000--,397.78
1,--ace--,166.82
2,--ionex--,99.93
3,-2SV-vuLB-Kg,427.50
4,-404PageNotFound-,1509.32
...,...,...
70907,zzonci,19.98
70908,zzoptimuszz,64.98
70909,zzydrax,99.94
70910,zzyfo,828.51


Para volver a agregar los items consumidos por cada usuario se hace un dataframe auxiliar 'df_count_items' conteniendo la cantidad de items por usuario.

In [62]:
df_count_items = df_gastos_items[['items_count', 'user_id']]
# se eliminan los duplicados
df_count_items = df_count_items.drop_duplicates(subset='user_id', keep='first')
df_count_items

,items_count,user_id
0,277,76561197970982479
277,888,js41637
1165,137,evcentric
1302,328,Riot-Punch
1630,541,doctr
...,...,...
5093574,321,76561198320136420
5093895,4,ArkPlays7
5093899,22,76561198323066619
5093921,177,76561198326700687


Finalmente, se juntan los dos dataframe auxiliares 'df_gastos_items_group' que contiene el gasto de cada usuario con 'df_count_items' que contiene la cantidad de items consumidos por cada usuario.

In [63]:
df_gastos_items = df_count_items.merge(df_gastos_items_group, on='user_id', how='right')
df_gastos_items

,items_count,user_id,price
0,58,--000--,397.78
1,44,--ace--,166.82
2,23,--ionex--,99.93
3,68,-2SV-vuLB-Kg,427.50
4,149,-404PageNotFound-,1509.32
...,...,...,...
70907,5,zzonci,19.98
70908,61,zzoptimuszz,64.98
70909,13,zzydrax,99.94
70910,84,zzyfo,828.51


## Playtime_forever por usuario

Se busca generar un dataframe que contenga por género de videojuego, el tiempo jugado por cada usuario con su id y url del perfil. Para ello, se realizarán algunos pasos generando algunos dataframes auxiliares.  
En primer lugar, se extrae de `df_items` las columnas 'playtime_forever', 'user_id' y 'item_id' y se reserva.

In [64]:
playtime_forever_usuario_item = df_items[['playtime_forever', 'user_id', 'item_id']]
playtime_forever_usuario_item

,playtime_forever,user_id,item_id
0,6,76561197970982479,10
1,0,76561197970982479,20
2,7,76561197970982479,30
3,0,76561197970982479,40
4,0,76561197970982479,50
...,...,...,...
5094100,0,76561198329548331,346330
5094101,0,76561198329548331,373330
5094102,3,76561198329548331,388490
5094103,4,76561198329548331,521570


Del dataframe `df_games` se extrae el 'id' del item y el género.

In [65]:
genre_item = df_games[['genres', 'id']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
genre_item = genre_item.rename(columns={'id':'item_id'})
genre_item

,genres,item_id
0,Action,761140
1,Casual,761140
2,Indie,761140
3,Simulation,761140
4,Strategy,761140
...,...,...
71546,Indie,610660
71547,Racing,610660
71548,Simulation,610660
71549,Casual,658870


Ahora se unen las dos tablas anteriores para obtener los generos de todos los usuarios con sus id y url.

In [66]:
df_playtime_genre = playtime_forever_usuario_item.merge(genre_item, on='item_id')
df_playtime_genre

,playtime_forever,user_id,item_id,genres
0,6,76561197970982479,10,Action
1,0,js41637,10,Action
2,0,Riot-Punch,10,Action
3,93,doctr,10,Action
4,108,corrupted_soul,10,Action
...,...,...,...,...
9877299,164,76561198107283457,354280,Indie
9877300,164,76561198107283457,354280,Simulation
9877301,0,inven,433920,Adventure
9877302,0,inven,433920,Indie


Se agrupa por género y usuario y se suma el tiempo de juego de para cada caso, se asume que la columna playtime_forever se encuentra en minutos, por lo que se transforma a horas.

In [67]:
# Se agrupa por usuario y se suma el tiempo de juego
agg_genero = df_playtime_genre.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()
# Se transforma la columna 'playtime_forever' a horas
agg_genero['playtime_horas'] = agg_genero['playtime_forever']/60
# Se borra la columna en minutos
agg_genero = agg_genero.drop('playtime_forever', axis=1)
agg_genero

,genres,user_id,playtime_horas
0,Action,--000--,2324.483333
1,Action,--ace--,1155.416667
2,Action,--ionex--,638.583333
3,Action,-2SV-vuLB-Kg,708.333333
4,Action,-404PageNotFound-,1957.050000
...,...,...,...
671349,Web Publishing,zepavil,632.100000
671350,Web Publishing,zeshirky,0.016667
671351,Web Publishing,zevlupine,0.066667
671352,Web Publishing,zilaman,0.150000


Se quiere tener el url del perfil del usuario, por lo que se hace un dataframe auxiliar del 'user_id' con 'user_url', se eliminan los duplicados para que queden usuarios sin repetir y finalmente se agrega al dataframe auxiliar anterior para generar finalmente el dataframe 'df_playtime_forever' que contiene por género las horas de juego por usuario y su url.

In [68]:
user_url_usuario = df_items[['user_url', 'user_id']]
# Se eliminan nuplicados para tener una sola vez los 'user_id' con su url
user_url_usuario= user_url_usuario.drop_duplicates(subset='user_id', keep='first')
# Se une con el dataframe agregado de género
df_playtime_forever = agg_genero.merge(user_url_usuario, on='user_id', how='left')
df_playtime_forever.head()

,genres,user_id,playtime_horas,user_url
0,Action,--000--,2324.483333,http://steamcommunity.com/id/--000--
1,Action,--ace--,1155.416667,http://steamcommunity.com/id/--ace--
2,Action,--ionex--,638.583333,http://steamcommunity.com/id/--ionex--
3,Action,-2SV-vuLB-Kg,708.333333,http://steamcommunity.com/id/-2SV-vuLB-Kg
4,Action,-404PageNotFound-,1957.050000,http://steamcommunity.com/id/-404PageNotFound-


## Ranking de géneros por tiempo de juego

En este punto lo que se busca es un dataframe auxiliar que contenga el ranking de los géneros de videojuegos en función del tiempo jugados de cada uno. Se aprovecha que ya se hizo para el punto anterior un dataframe auxiliar que contiene los géneros y el tiempo jugado, por lo que se usa el mismo. Primero se agrupa por género, luego se ordena y se agrega una columna de ranking.

In [69]:
df_genre_ranking = agg_genero.groupby('genres')['playtime_horas'].sum().reset_index()
# Se ordena por 'playtime_horas'
df_genre_ranking = df_genre_ranking.sort_values(by='playtime_horas', ascending=False)
# Agregar una columna con la posición del ranking
df_genre_ranking['ranking'] = df_genre_ranking['playtime_horas'].rank(ascending=False).astype(int)
df_genre_ranking

,genres,playtime_horas,ranking
0,Action,5.125167e+07,1
9,Indie,2.459123e+07,2
12,RPG,1.713213e+07,3
1,Adventure,1.497939e+07,4
14,Simulation,1.425438e+07,5
17,Strategy,1.084995e+07,6
8,Free to Play,1.006072e+07,7
10,Massively Multiplayer,7.352093e+06,8
4,Casual,4.155277e+06,9
6,Early Access,2.611542e+06,10


## Cantidad de items y contenido free por desarrollador

En este punto se busca un dataframe que contenga para cada desarrollador de juegos, los items que desarrolla, el año de lanzamiento y el precio de cada uno.

In [70]:
price_anio_developer = df_games[['price', 'release_anio', 'developer', 'id']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
price_anio_developer = price_anio_developer.rename(columns={'id':'item_id'})
# se eliminan los duplicados
df_items_developer = price_anio_developer.drop_duplicates()
df_items_developer

,price,release_anio,developer,item_id
0,4.99,2018,Kotoshiro,761140
5,0.00,2018,Secret Level SRL,643980
9,0.00,2017,Poolians.com,670290
14,0.99,2017,彼岸领域,767400
17,3.99,2018,Trickjump Games Ltd,772540
...,...,...,...,...
71535,1.99,2018,Bidoniera Games,745400
71539,1.99,2018,"Nikita ""Ghost_RUS""",773640
71543,4.99,2018,Sacada,733530
71546,1.99,2018,Laush Dmitriy Sergeevich,610660


## 'release_anio' en `df_reviews`

En este punto se agrega el año de lanzamiento de un juego al dataframe `df_reviews`. En primer lugar se extraen las columnas de id del juego y el año de lanzamiento del mismo, se borran los duplicados y luego se unen con el dataframe de reviews.

In [71]:
anio_lanzamiento_item = df_games[['id', 'release_anio']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
anio_lanzamiento_item = anio_lanzamiento_item.rename(columns={'id':'reviews_item_id'})
# se eliminan los duplicados
anio_lanzamiento_item = anio_lanzamiento_item.drop_duplicates()
anio_lanzamiento_item

,reviews_item_id,release_anio
0,761140,2018
5,643980,2018
9,670290,2017
14,767400,2017
17,772540,2018
...,...,...
71535,745400,2018
71539,773640,2018
71543,733530,2018
71546,610660,2018


In [72]:
df_reviews = df_reviews.merge(anio_lanzamiento_item, on='reviews_item_id')
df_reviews

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,release_anio
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,2009
1,EndAtHallow,http://steamcommunity.com/id/EndAtHallow,1250,No ratings yet,True,2015-01-15,1,2009
2,76561198077432581,http://steamcommunity.com/profiles/76561198077...,1250,No ratings yet,True,2014-12-12,1,2009
3,76561198057958244,http://steamcommunity.com/profiles/76561198057...,1250,0 of 1 people (0%) found this review helpful,True,2013-12-13,0,2009
4,46366536564574576346346546,http://steamcommunity.com/id/46366536564574576...,1250,2 of 3 people (67%) found this review helpful,True,2014-08-19,1,2009
...,...,...,...,...,...,...,...,...
48797,ButtBurger2,http://steamcommunity.com/id/ButtBurger2,73010,No ratings yet,True,2012-05-17,0,2011
48798,76561198064526566,http://steamcommunity.com/profiles/76561198064...,378930,3 of 17 people (18%) found this review helpful,False,Formato inválido,1,2016
48799,haungaraho,http://steamcommunity.com/id/haungaraho,16600,No ratings yet,True,2012-10-22,2,2008
48800,UnseenPrecision,http://steamcommunity.com/id/UnseenPrecision,232950,No ratings yet,True,2014-01-19,1,2013


## Carga de los dataframe

In [73]:
dfs = [df_reviews, df_games, df_items, df_gastos_items, df_genre_ranking, df_playtime_forever, df_items_developer]
# Nombres correspondientes a cada DataFrame
names = ['df_reviews', 'df_games', 'df_items', 'df_gastos_items', 'df_genre_ranking', 'df_playtime_forever', 'df_items_developer']

for df, name in zip(dfs, names):
    archivo = f'data/{name}_unido.csv'
    df.to_csv(archivo, index=False, encoding='utf-8')
    print(f"DataFrame '{name}' guardado como '{archivo}'")

DataFrame 'df_reviews' guardado como 'data/df_reviews_unido.csv'
DataFrame 'df_games' guardado como 'data/df_games_unido.csv'
DataFrame 'df_items' guardado como 'data/df_items_unido.csv'
DataFrame 'df_gastos_items' guardado como 'data/df_gastos_items_unido.csv'
DataFrame 'df_genre_ranking' guardado como 'data/df_genre_ranking_unido.csv'
DataFrame 'df_playtime_forever' guardado como 'data/df_playtime_forever_unido.csv'
DataFrame 'df_items_developer' guardado como 'data/df_items_developer_unido.csv'


Para optimizar la estructura de los datos en el deploy, se aprovecha en este punto a guardar los dataframe en formato parquet.

In [77]:
for df, name in zip(dfs, names):
    archivo = f'data/{name}.parquet'
    pq.write_table(pa.Table.from_pandas(df), archivo)
    print(f"DataFrame '{name}' guardado como '{archivo}'")

DataFrame 'df_reviews' guardado como 'data/df_reviews.parquet'
DataFrame 'df_games' guardado como 'data/df_games.parquet'
DataFrame 'df_items' guardado como 'data/df_items.parquet'
DataFrame 'df_gastos_items' guardado como 'data/df_gastos_items.parquet'
DataFrame 'df_genre_ranking' guardado como 'data/df_genre_ranking.parquet'
DataFrame 'df_playtime_forever' guardado como 'data/df_playtime_forever.parquet'
DataFrame 'df_items_developer' guardado como 'data/df_items_developer.parquet'
